In [1]:
import os
import time

import requests
from tqdm import tqdm

In [10]:
def download_file_with_retry(
    url, destination, chunk_size=1024, max_retries=15, retry_delay=60, timeout=60
):
    retry_count = 0
    resume_header = {}
    if os.path.exists(destination):
        resume_header["Range"] = "bytes=%d-" % os.path.getsize(destination)

    while retry_count < max_retries:
        try:
            with requests.get(
                url, stream=True, headers=resume_header, timeout=timeout
            ) as response:
                response.raise_for_status()

                file_size = int(response.headers.get("content-length", 0))

                with open(destination, "ab") as file, tqdm(
                    total=file_size,
                    unit="B",
                    unit_scale=True,
                    unit_divisor=1024,
                    desc=os.path.basename(destination),
                ) as progress_bar:
                    for chunk in response.iter_content(chunk_size=chunk_size):
                        if chunk:
                            file.write(chunk)
                            progress_bar.update(len(chunk))

            print("Download complete!")

            return True

        except requests.exceptions.RequestException as e:
            print(f"Error occurred: {e}")

            retry_count += 1

            if retry_count < max_retries:
                print(f"Retrying download in {retry_delay} seconds...")
                time.sleep(retry_delay)

    print("Max retries reached. Download failed.")

    return False

In [11]:
# Example usage
file = "PointCloud_pt_zips/compressed_pcs_pt_06.zip"
url = f"https://huggingface.co/datasets/tiange/Cap3D/resolve/main/{file}?download=true"

destination = file

download_file_with_retry(url, destination, chunk_size=8192)

Error occurred: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /datasets/tiange/Cap3D/resolve/main/PointCloud_pt_zips/compressed_pcs_pt_06.zip?download=true (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x728ef9913c70>: Failed to resolve 'huggingface.co' ([Errno -3] Temporary failure in name resolution)"))
Retrying download in 60 seconds...


compressed_pcs_pt_06.zip:   0%|          | 13.8M/3.97G [01:15<6:07:10, 193kB/s] 


Error occurred: HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Read timed out.
Retrying download in 60 seconds...


compressed_pcs_pt_06.zip: 100%|██████████| 3.97G/3.97G [1:54:53<00:00, 619kB/s]   

Download complete!


True